# IF5200 - Evaluation Notebook
___
Group: 8<br>
Project: Automated Chest X-Ray Report Generator in Bahasa Indonesia with the Use of Deep Learning<br>
Team members: Arief Purnama Muharram, Hollyana Puteri Haryono, Abassi Haji Juma

Last update: April 5th, 2023

## A. Print library version

In [1]:
print('tqdm version:', __import__('tqdm').__version__)
print('matplotlib version:', __import__('matplotlib').__version__)
print('seaborn version:', __import__('seaborn').__version__)
print('pandas version:', __import__('pandas').__version__)
print('scikit-learn version:', __import__('sklearn').__version__)
print('imblearn version:', __import__('imblearn').__version__)
print('pillow version:', __import__('PIL').__version__)
print('torch version:', __import__('torch').__version__)
print('torchvision version:', __import__('torchvision').__version__)

tqdm version: 4.64.1
matplotlib version: 3.5.3
seaborn version: 0.12.2
pandas version: 1.3.5
scikit-learn version: 1.0.2
imblearn version: 0.10.1
pillow version: 9.4.0
torch version: 1.13.1+cu117
torchvision version: 0.14.1+cu117


## B. Load all helpers

In [2]:
from src.report.utils.inference import InferenceUtils
from src.utils.image import read_image, get_segment, ToTensorTransform

## C. Load dataset

In [3]:
import pandas as pd


df = pd.read_csv('datasets/labels_cxr-images.csv', sep=',')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112120 entries, 0 to 112119
Data columns (total 18 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   Image Index         112120 non-null  object
 1   Filename_Segment1   112120 non-null  object
 2   Filename_Segment2   112120 non-null  object
 3   Filename_Segment3   112120 non-null  object
 4   Atelectasis         112120 non-null  int64 
 5   Cardiomegaly        112120 non-null  int64 
 6   Effusion            112120 non-null  int64 
 7   Infiltration        112120 non-null  int64 
 8   Mass                112120 non-null  int64 
 9   Nodule              112120 non-null  int64 
 10  Pneumonia           112120 non-null  int64 
 11  Pneumothorax        112120 non-null  int64 
 12  Consolidation       112120 non-null  int64 
 13  Edema               112120 non-null  int64 
 14  Emphysema           112120 non-null  int64 
 15  Fibrosis            112120 non-null  int64 
 16  Pl

In [4]:
# Select only portions of data for system evaluation
data = df[['Image Index', 'Cardiomegaly', 'Effusion']]
data = data.rename(columns={'Image Index': 'filename', 'Cardiomegaly': 'cardiomegaly', 'Effusion': 'effusion'}, errors='ignore')
data = data.loc[~((data['cardiomegaly'] == 0) & (data['effusion'] == 0))]
data = data.sample(frac=1).head(1000)
data.reset_index(inplace=True)
data.drop(['index'], axis=1, inplace=True)

In [5]:
data.head(3)

,filename,cardiomegaly,effusion
0,00012649_007.png,0,1
1,00014387_009.png,0,1
2,00004533_010.png,1,0


## D. Make evaluation

In [6]:
MODELS_DIR = 'sys/models'
IMAGES_DIR = 'datasets/data_cxr-images_128x128'

In [7]:
import os
import torch


# Setup device
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# Load models
model_cardiomegaly = torch.load(os.path.join(MODELS_DIR, 'model_cardiomegaly.pth'), map_location=device)
model_effusion = torch.load(os.path.join(MODELS_DIR, 'model_effusion.pth'), map_location=device)
# -- Define another models here, if necessary --

# Setup inferences
inference_cardiomegaly = InferenceUtils(model=model_cardiomegaly, device=device)
inference_effusion = InferenceUtils(model=model_effusion, device=device)
# -- Define another inferences here, if necessary --


In [8]:
import os


# Prediction function
def make_prediction(filename):
    image_path = os.path.join(IMAGES_DIR, filename)
    image = read_image(image_path)
    
    # Slice the image
    img_segment1 = get_segment(image, 1)
    img_segment2 = get_segment(image, 2)
    img_segment3 = get_segment(image, 3)
    
    # Convert to tensors
    transformer = ToTensorTransform()
    img_segment1 = transformer.transform(img_segment1)
    img_segment2 = transformer.transform(img_segment2)
    img_segment3 = transformer.transform(img_segment3)
    
    # Make prediction
    result_cardiomegaly = inference_cardiomegaly.make_prediction(img_segment2.unsqueeze(0)).item()
    result_effusion = inference_effusion.make_prediction(img_segment2.unsqueeze(0)).item()
    # -- Make another prediction here, if necessary --
    
    # Return result
    # return [result_1, result_2, ..., result_n]
    return [result_cardiomegaly, result_effusion]
    

In [9]:
import os
from time import sleep
from tqdm import tqdm
from pandas import DataFrame


# Evaluate function
def evaluate(data: DataFrame) -> DataFrame:
    result = []
    
    for idx, row in tqdm(data.iterrows(), total=data.shape[0]):
        # ground_truth = [ground_truth_1, ground_truth_2, ..., ground_truth_n]
        ground_truth = [row['cardiomegaly'], row['effusion']]
        result_code = make_prediction(row['filename'])
        
        # result.append([row['filename'], row['label_1'], row['label_2'], ..., row['label_n'], result_code == ground_truth])
        result.append([row['filename'], row['cardiomegaly'], row['effusion'], result_code[0], result_code[1], result_code == ground_truth])
        
    result = DataFrame(result, columns=['filename', 'cardiomegaly', 'effusion', 'pred_cardiomegaly', 'pred_effusion', 'result'])
    
    size = len(result)
    count_true = len(result[result['result'] == True])
    
    sleep(3)
    
    print('\nEvaluation report')
    print('-' * 21)
    print('Total data:', size)
    print('Count true:', count_true)
    print('Accuracy:', round((count_true / size), 2), '\n')
    
    return result
    

In [10]:
# Make evaluation
result = evaluate(data)

100%|██████████| 1000/1000 [03:21<00:00,  4.97it/s]



Evaluation report
---------------------
Total data: 1000
Count true: 1
Accuracy: 0.0 



In [11]:
# Print result
result.head(15)

,filename,cardiomegaly,effusion,pred_cardiomegaly,pred_effusion,result
0,00012649_007.png,0,1,0,0,False
1,00014387_009.png,0,1,0,0,False
2,00004533_010.png,1,0,0,0,False
3,00010222_018.png,0,1,0,0,False
4,00013695_005.png,0,1,0,0,False
5,00019707_012.png,0,1,0,0,False
6,00000013_021.png,0,1,0,0,False
7,00021840_011.png,0,1,0,0,False
8,00018187_055.png,0,1,0,0,False
9,00011237_094.png,0,1,0,0,False


In [12]:
# Error analysis
result[result['result'] == True].head(15)

,filename,cardiomegaly,effusion,pred_cardiomegaly,pred_effusion,result
753,00010994_007.png,1,1,1,1,True


In [13]:
# Error analysis
result[result['result'] == False].head(15)

,filename,cardiomegaly,effusion,pred_cardiomegaly,pred_effusion,result
0,00012649_007.png,0,1,0,0,False
1,00014387_009.png,0,1,0,0,False
2,00004533_010.png,1,0,0,0,False
3,00010222_018.png,0,1,0,0,False
4,00013695_005.png,0,1,0,0,False
5,00019707_012.png,0,1,0,0,False
6,00000013_021.png,0,1,0,0,False
7,00021840_011.png,0,1,0,0,False
8,00018187_055.png,0,1,0,0,False
9,00011237_094.png,0,1,0,0,False
